<a href="https://colab.research.google.com/github/Duchai263/QAbot/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [2]:
%pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   

In [3]:
!pip install -qU langchain-huggingface

In [4]:
!pip install -qU langchain-community

In [5]:
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.5 MB/s eta 0:00:00


In [6]:
!pip install flask
!pip install pyngrok

# Retrieval

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name= "TinyLlama/TinyLlama-1.1B-Chat-v1.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("Init vector")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [3]:
from langchain_core.documents import Document

def add_document(doc: Document):
  vector_store.add_documents([doc])

def get_document(query: str):
  return vector_store.similarity_search(query, k=1)

## test

In [4]:
with open("policies.txt","r",encoding="utf-8") as f:
  context = f.read()
  doc = Document(page_content=context, metadata={"source": "policies.txt"})
  add_document(doc)

In [4]:
from uuid import uuid4
from langchain_core.documents import Document

docs_path = ["policies1.txt","policies2.txt","policies3.txt"]
documents = []
for doc in docs_path:
  with open(doc,"r",encoding="utf-8") as f:
    documents.append(Document(page_content=f.read(), metadata={"source": doc}))

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['fb7fed5f-bf96-4c8c-9ee1-1de7ad63b191',
 '5bcea8df-882b-4bf3-943a-a09e66c76494',
 '1f212aaa-81cb-42cb-a482-58a9218228ad']

In [6]:
question = "Can I come late at 11 AM?"

results = vector_store.similarity_search(
    question,
    k=1
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Working Time
Working Time
From 9.00 AM- 6.00 PM, Monday – Friday ( or 8.00 AM- 5.00 PM, as long as you can ensure 8 hours of work per day)
Lunch break for 01 hour (from 12:00 PM to 1:00 PM)
Possibility to flex before 10.00 AM and after 4.00 PM. The team and engineering coach must be informed if flex time is taken between 10.00 AM to 4.00 PM
You can go to the office on  the weekend for work but you need to inform your team and Eng. Coach on Friday as the latest before the weekend.   Employees who would like to work at the office on Sunday have to inform Mrs. Linh before 4.00 PM on Friday to register with the BOM of the PJICO Building.
 [{'source': 'policies1.txt'}]


# Create model

In [4]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate

# model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# model_id = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
                do_sample=False,
                temperature=None,
                top_p=None,
                repetition_penalty=1.2,
                # add_generation_prompt=False
                max_new_tokens=200,
                )
llm = HuggingFacePipeline(pipeline=pipe)

template = """You are a business assistant. Provide a answer to the employee's question with given context.
context: {context}

question: {question}

Answer:
"""
prompt = PromptTemplate.from_template(template)
chain = prompt | llm.bind(skip_prompt=True)


def get_answer(question: str):
  results = vector_store.similarity_search(question,k=1)

  context = results[0].page_content
  soucrce = results[0].metadata["source"]

  answer = chain.invoke({"context": context, "question": question})
  return {"answer": answer, "source": soucrce}

Device set to use cuda:0


## test

In [10]:
question = "what is the Working Time"

print(f"Question: {question} ")
print(f"\nModel name: {model_id}")

response = get_answer(question)
answer = response["answer"]
source = response["source"]

print(f"Answer: {answer}")
print(f"Source: {source}")

Question: what is the Working Time 

Model name: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Answer: Working time means the number of hours an individual works or spends on work activities. It includes all types of work such as office work, manual labor, and other non-office related tasks. The term ‘working time’ refers to the total amount of time that an employee has been assigned by their employer to perform specific duties within a particular organization. In our example, the working time is 8 hours per day, which can be divided into two parts – morning and afternoon shifts.
Source: policies2.txt


In [2]:
with open("policies1.txt","r",encoding="utf-8") as f:
  context = f.read()
print(context)

Working Time
Working Time
From 9.00 AM- 6.00 PM, Monday – Friday ( or 8.00 AM- 5.00 PM, as long as you can ensure 8 hours of work per day)
Lunch break for 01 hour (from 12:00 PM to 1:00 PM)
Possibility to flex before 10.00 AM and after 4.00 PM. The team and engineering coach must be informed if flex time is taken between 10.00 AM to 4.00 PM
You can go to the office on  the weekend for work but you need to inform your team and Eng. Coach on Friday as the latest before the weekend.   Employees who would like to work at the office on Sunday have to inform Mrs. Linh before 4.00 PM on Friday to register with the BOM of the PJICO Building.



In [3]:
from langchain_core.prompts import PromptTemplate
question = "Can I come late at 11 AM?"

response = chain.invoke({"context": context, "question": question})

print(f"Question: {question} ")
print(f"\nModel name: {model_id}")
print(f"Answer: {response}")

Question: Can I come late at 11 AM? 

Model name: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Answer: Yes, you may come late at 11 AM provided that you notify your supervisor in advance via email or phone call. If you arrive later than this time slot without prior notice, it will result in an unscheduled absence from work.


# Flask app - ngrok

In [7]:
import getpass
import os
import threading

from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


def get_ai_response(user_input: str) -> str:
    response = get_answer(user_input)
    answer = response["answer"]
    source = response["source"]

    return f"{answer} \n Source: {source}"

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    response = get_ai_response(user_input)
    return jsonify({'response': response})

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    if file.filename == '' or not file.filename.lower().endswith('.txt'):
        return jsonify({'error': 'Invalid file format. Only .txt files are allowed.'}), 400


    file_content = file.read().decode('utf-8')
    file_name = file.filename

    doc = Document(page_content=file_content, metadata={"source": file_name})
    add_document(doc)

    return jsonify({'message': 'File content read successfully.', 'content': file_content, 'filename': file_name})
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
# app.run()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········
 * ngrok tunnel "https://5a7e-34-124-151-240.ngrok-free.app" -> "http://127.0.0.1:5000"


In [18]:
!ps -eaf | grep 5000

root        4766    1126  0 06:37 ?        00:00:00 /bin/bash -c ps -eaf | grep 5000
root        4768    4766  0 06:37 ?        00:00:00 grep 5000
